In [1]:
import gc
import matplotlib.pyplot as plt
plt.close('all')
gc.collect()
print("Memoria liberada.")

Memoria liberada.


In [2]:
# =========================
# Cell 1 - Configuración inicial y carga de datos (Batch1 Only)
# =========================

import os
import numpy as np
from tqdm import tqdm
import nibabel as nib

# Limitar hilos de BLAS/OpenMP (para evitar sobrecarga en CPU)
os.environ["OMP_NUM_THREADS"] = "1"
os.environ["OPENBLAS_NUM_THREADS"] = "1"
os.environ["MKL_NUM_THREADS"] = "1"
os.environ["NUMEXPR_NUM_THREADS"] = "1"

# -------------------------
# Global parameters (HU Clipping alineado con roi.ipynb)
# -------------------------
HU_MIN, HU_MAX = -1024, 600

# -------------------------
# Database paths (SOLO TrainBatch1)
# -------------------------
# Se usan rutas absolutas como en tu entorno original de trabajo
DATA_PATHS = {
    "images_batch1": r"./TrainBatch1/imagesTr",
    "labels_batch1": r"./TrainBatch1/labelsTr",
    # "images_batch2": r"./TrainBatch2/imagesTr",  # Comentado: no está disponible
    # "labels_batch2": r"./TrainBatch2/labelsTr"   # Comentado: no está disponible
}

# Validar solo las rutas existentes
for key in ["images_batch1", "labels_batch1"]:
    path = DATA_PATHS[key]
    assert os.path.exists(path), f"Path not found: {path}"

# -------------------------
# Count files
# -------------------------
counts = {}
for key in ["images_batch1", "labels_batch1"]:
    path = DATA_PATHS[key]
    files = [f for f in os.listdir(path) if f.endswith(".nii.gz")]
    counts[key] = len(files)

print(">> Configuration loaded.")
print(f"Batch1 imagesTr: {counts['images_batch1']} archivos")
print(f"Batch1 labelsTr: {counts['labels_batch1']} archivos")
print(f"TOTAL archivos en la base: {sum(counts.values())}")

# -------------------------
# Build combined list of volumes and labels (SOLO Batch1)
# -------------------------
all_pairs = []
# Solo procesar batch1
batch = "batch1"
images_path = DATA_PATHS[f"images_{batch}"]
labels_path = DATA_PATHS[f"labels_{batch}"]
image_files = [f for f in os.listdir(images_path) if f.endswith(".nii.gz")]
for img_file in image_files:
    base_id = img_file.replace("_0000.nii.gz", "")
    candidates = [f for f in os.listdir(labels_path) if base_id in f]
    if candidates:
        label_file = candidates[0]
        all_pairs.append((os.path.join(images_path, img_file),
                          os.path.join(labels_path, label_file)))
assert len(all_pairs) > 0, "No image-label pairs found in TrainBatch1."

# -------------------------
# Index all volumes (preserve original metadata)
# -------------------------
loaded_data = {}
for img_path, lbl_path in tqdm(all_pairs, desc="Indexing volumes from TrainBatch1"):
    img_obj = nib.load(img_path, mmap=True)
    vol_id = os.path.basename(img_path).replace("_0000.nii.gz", "")
    # ✅ Guardamos affine y header originales sin modificarlos
    loaded_data[vol_id] = {
        "img_path": img_path,
        "lbl_path": lbl_path,
        "affine": img_obj.affine,
        "header": img_obj.header.copy(),
        "zooms": tuple(img_obj.header.get_zooms()[:3])
    }
    img_obj.uncache()  # liberar mmap
    del img_obj

print(f">> Indexed {len(loaded_data)} volumes from TrainBatch1.")

>> Configuration loaded.
Batch1 imagesTr: 150 archivos
Batch1 labelsTr: 150 archivos
TOTAL archivos en la base: 300


Indexing volumes from TrainBatch1: 100%|██████████| 150/150 [00:00<00:00, 597.72it/s]

>> Indexed 150 volumes from TrainBatch1.


In [3]:
# =========================
#   Procesamiento masivo — ALINEADO CON Cell 2-3 (búsqueda multi-eje)
# =========================

import os
import gc
import numpy as np
import nibabel as nib
from tqdm import tqdm
from skimage.morphology import remove_small_objects, binary_closing, binary_dilation, ball
from skimage.segmentation import clear_border
from scipy.ndimage import label as cc_label, binary_fill_holes
from collections import deque

# --- Rutas ---
OUTPUT_DIR = r"."  
ROIS_SUBDIR = os.path.join(OUTPUT_DIR, "rois_auto")
os.makedirs(ROIS_SUBDIR, exist_ok=True)

# --- Parámetros (¡IGUALES que en roi.ipynb!) ---
AIR_THR_STRICT_ROI = -950
GROW_THR_ROI = -700
MIN_SIZE_ROI = 50          # ✅ Alineado con Cell 2-3
SMOOTH_RADIUS_ROI = 5      # ✅ Alineado con Cell 2-3
HU_MIN, HU_MAX = -1024, 600

# --- Verificar volúmenes existentes ---
existing = {f for f in os.listdir(ROIS_SUBDIR) if f.endswith(".nii.gz")}
to_process = [vid for vid in loaded_data.keys() if f"{vid}.nii.gz" not in existing]
print(f"🔄 Procesando {len(to_process)} volúmenes con pipeline de roi.ipynb (búsqueda multi-eje)...")

# --- Procesamiento masivo ---
for vol_id in tqdm(to_process, desc="Generando ROIs"):
    try:
        # --- Cargar volumen ---
        data = loaded_data[vol_id]
        vol_orig = nib.load(data["img_path"]).get_fdata(dtype=np.float32)
        affine = data["affine"]
        header = data["header"]
        vol_clip = np.clip(vol_orig, HU_MIN, HU_MAX)
        Z, Y, X = vol_clip.shape

        # --- Paso 1: Semillas con búsqueda multi-eje (¡MISMA LÓGICA que Cell 2-3!) ---
        seeds = (vol_clip <= AIR_THR_STRICT_ROI)
        seeds = clear_border(seeds)
        seeds = remove_small_objects(seeds, min_size=MIN_SIZE_ROI)

        # Regiones de búsqueda para cada eje
        delta_z = Z // 10
        delta_y = Y // 8
        delta_x = X // 8

        # Axial: alrededor de Z//2
        axial_region = np.zeros_like(seeds, dtype=bool)
        z_axial = Z // 2
        axial_region[max(0, z_axial - delta_z):min(Z, z_axial + delta_z), :, :] = True

        # Coronal: alrededor de Y//2
        coronal_region = np.zeros_like(seeds, dtype=bool)
        y_coronal = Y // 2
        coronal_region[:, max(0, y_coronal - delta_y):min(Y, y_coronal + delta_y), :] = True

        # Sagittal: alrededor de X//2
        sagittal_region = np.zeros_like(seeds, dtype=bool)
        x_sagittal = X // 2
        sagittal_region[:, :, max(0, x_sagittal - delta_x):min(X, x_sagittal + delta_x)] = True

        # Intersección de regiones
        search_region = axial_region & coronal_region & sagittal_region
        trachea_cand = seeds & search_region

        # Detección de tráquea
        trachea = np.zeros_like(seeds, dtype=bool)
        labeled_top, n_top = cc_label(trachea_cand)
        if n_top > 0:
            sizes = np.bincount(labeled_top.ravel())
            sizes[0] = 0
            main_label = np.argmax(sizes)
            trachea = (labeled_top == main_label)

        # --- Paso 2: BFS optimizado (6-conectado) ---
        airways_mask = np.zeros_like(seeds, dtype=np.uint8)
        seed_pts = np.argwhere(trachea)

        if len(seed_pts) > 0:
            q = deque(seed_pts[:100])  # Limitar semillas iniciales
            for pt in seed_pts[:100]:
                airways_mask[tuple(pt)] = 1

            neighbors = [(1,0,0), (-1,0,0), (0,1,0), (0,-1,0), (0,0,1), (0,0,-1)]
            while q:
                z, y, x = q.popleft()
                for dz, dy, dx in neighbors:
                    nz, ny, nx = z + dz, y + dy, x + dx
                    if (0 <= nz < Z and 0 <= ny < Y and 0 <= nx < X and 
                        airways_mask[nz, ny, nx] == 0 and vol_clip[nz, ny, nx] <= GROW_THR_ROI):
                        airways_mask[nz, ny, nx] = 1
                        q.append((nz, ny, nx))

        # --- Paso 3: Componente conectada ---
        labeled_airways, n_airways = cc_label(airways_mask)
        if n_airways > 0 and len(seed_pts) > 0:
            sz, sy, sx = seed_pts[0]
            seed_label = labeled_airways[sz, sy, sx]
            airways_mask = (labeled_airways == seed_label).astype(np.uint8)

        # --- Paso 4: Post-procesamiento (¡MISMO que Cell 2-3!) ---
        mask_clean = remove_small_objects(airways_mask.astype(bool), min_size=MIN_SIZE_ROI)
        mask_closed = binary_closing(mask_clean, footprint=ball(SMOOTH_RADIUS_ROI))
        mask_filled = binary_fill_holes(mask_closed)
        mask_closed3d = binary_closing(mask_filled, footprint=ball(2))  # Usar skimage.binary_closing
        mask_dilated = binary_dilation(mask_closed3d, footprint=ball(1))
        final_roi = binary_fill_holes(mask_dilated).astype(np.uint8)

        # --- Eliminar fondo de forma robusta ---
        background_mask = (vol_clip <= -1000)
        final_roi[background_mask] = 0

        # --- Guardar ---
        output_path = os.path.join(ROIS_SUBDIR, f"{vol_id}.nii.gz")
        nib.save(nib.Nifti1Image(final_roi, affine, header), output_path)

        # --- Limpieza de memoria ---
        del vol_orig, vol_clip, final_roi
        gc.collect()

    except Exception as e:
        print(f"\n❌ Error en {vol_id}: {e}")
        gc.collect()

print("✅ ROIs generadas para todos los volúmenes. Pipeline fiel a Cell 2-3 (búsqueda multi-eje).")

🔄 Procesando 0 volúmenes con pipeline de roi.ipynb (búsqueda multi-eje)...


Generando ROIs: 0it [00:00, ?it/s]

✅ ROIs generadas para todos los volúmenes. Pipeline fiel a Cell 2-3 (búsqueda multi-eje).


In [4]:
# =========================
# Cell 4: Segmentación masiva de vías aéreas + métricas (Versión TUBULAR - SIN MEMORY ERROR)
# =========================

import os
import time
import gc
import csv
import numpy as np
import nibabel as nib
from tqdm import tqdm
from skimage.morphology import ball
from scipy.ndimage import label as cc_label, binary_dilation, binary_closing
from collections import deque

# --- Rutas ---
BASE_OUTPUT_DIR = r"."
ROIS_DIR = os.path.join(BASE_OUTPUT_DIR, "rois_auto")
AIRWAYS_SUBDIR = os.path.join(BASE_OUTPUT_DIR, "airways_auto")
METRICS_FILE = os.path.join(BASE_OUTPUT_DIR, "airways_metrics.csv")

os.makedirs(AIRWAYS_SUBDIR, exist_ok=True)

# --- Parámetros ---
HU_MIN, HU_MAX = -1024, 600

# --- Función de carga eficiente ---
def load_volume_efficient(nii_path, dtype=np.float32):
    """Carga volumen sin mantener todo en memoria innecesariamente."""
    img = nib.load(nii_path)
    # Acceder directamente al dataobj evita copias innecesarias
    vol = np.asarray(img.dataobj, dtype=dtype)
    return vol, img.affine, img.header

def load_label_efficient(nii_path):
    """Carga label como uint8 para ahorrar memoria."""
    img = nib.load(nii_path)
    label = np.asarray(img.dataobj, dtype=np.uint8)
    return label

# --- Función de segmentación TUBULAR optimizada ---
def segment_airways_tubular(vol_clip, final_roi):
    """Segmenta vías aéreas usando la lógica TUBULAR robusta."""
    Z, Y, X = vol_clip.shape
    
    roi_mask = (final_roi > 0)
    if not np.any(roi_mask):
        return np.zeros_like(vol_clip, dtype=np.uint8), 0
    
    roi_hu_values = vol_clip[roi_mask]
    if len(roi_hu_values) == 0:
        return np.zeros_like(vol_clip, dtype=np.uint8), 0

    GROW_THR = np.percentile(roi_hu_values, 3)
    AIR_THR_STRICT = min(-980, np.percentile(roi_hu_values, 0.5))

    # --- Búsqueda multi-eje para tráquea ---
    seeds = (vol_clip <= AIR_THR_STRICT) & roi_mask

    delta_z = Z // 10
    delta_y = Y // 8  
    delta_x = X // 8

    axial_region = np.zeros_like(seeds, dtype=bool)
    axial_region[max(0, Z//2 - delta_z):min(Z, Z//2 + delta_z), :, :] = True

    coronal_region = np.zeros_like(seeds, dtype=bool)
    coronal_region[:, max(0, Y//2 - delta_y):min(Y, Y//2 + delta_y), :] = True

    sagittal_region = np.zeros_like(seeds, dtype=bool)
    sagittal_region[:, :, max(0, X//2 - delta_x):min(X, X//2 + delta_x)] = True

    search_region = axial_region & coronal_region & sagittal_region
    trachea_candidates = seeds & search_region

    labeled_trachea, num_components = cc_label(trachea_candidates)
    if num_components == 0:
        return np.zeros_like(vol_clip, dtype=np.uint8), 0

    component_sizes = np.bincount(labeled_trachea.ravel())
    component_sizes[0] = 0
    if np.all(component_sizes == 0):
        return np.zeros_like(vol_clip, dtype=np.uint8), 0

    main_trachea_label = np.argmax(component_sizes)
    trachea = (labeled_trachea == main_trachea_label)
    seed_points = np.argwhere(trachea)

    if len(seed_points) == 0:
        return np.zeros_like(vol_clip, dtype=np.uint8), 0

    # --- BFS estricto ---
    airways_bfs = np.zeros_like(seeds, dtype=np.uint8)
    queue = deque(seed_points[:200])  # Limitar semillas iniciales
    
    for pt in seed_points[:200]:
        airways_bfs[tuple(pt)] = 1

    def get_neighbors(z, y, x, shape):
        for dz, dy, dx in [(1,0,0), (-1,0,0), (0,1,0), (0,-1,0), (0,0,1), (0,0,-1)]:
            nz, ny, nx = z + dz, y + dy, x + dx
            if 0 <= nz < shape[0] and 0 <= ny < shape[1] and 0 <= nx < shape[2]:
                yield nz, ny, nx

    while queue:
        z, y, x = queue.popleft()
        for nz, ny, nx in get_neighbors(z, y, x, airways_bfs.shape):
            if (airways_bfs[nz, ny, nx] == 0 and 
                final_roi[nz, ny, nx] > 0 and 
                vol_clip[nz, ny, nx] <= GROW_THR):
                airways_bfs[nz, ny, nx] = 1
                queue.append((nz, ny, nx))

    # Post-procesamiento
    airways_mask = airways_bfs.astype(bool)
    airways_mask = binary_dilation(airways_mask, iterations=1)
    airways_mask = binary_closing(airways_mask, structure=ball(2))

    # --- Filtrado tubular ---
    labeled_mask, n_comp = cc_label(airways_mask)
    if n_comp == 0:
        return airways_mask.astype(np.uint8), len(seed_points)
    
    comp_sizes = np.bincount(labeled_mask.ravel())
    comp_sizes[0] = 0
    if len(comp_sizes) <= 1 or np.all(comp_sizes == 0):
        return airways_mask.astype(np.uint8), len(seed_points)
    
    main_comp_id = np.argmax(comp_sizes)
    valid_mask = (labeled_mask == main_comp_id)
    
    if n_comp > 1:
        for i in range(1, n_comp + 1):
            if i == main_comp_id:
                continue
            comp = (labeled_mask == i)
            size = np.sum(comp)
            if size < 30 or size > 50000:
                continue
            coords = np.argwhere(comp)
            if coords.size == 0:
                continue
            try:
                z_min, y_min, x_min = coords.min(axis=0)
                z_max, y_max, x_max = coords.max(axis=0)
                dims = np.sort([max(1, z_max-z_min), max(1, y_max-y_min), max(1, x_max-x_min)])
                if dims[2] >= 2 * dims[0]:
                    valid_mask = valid_mask | comp
            except:
                continue
    
    airways_mask = valid_mask

    # --- Extensión iterativa ---
    extended = airways_mask.copy()
    frontier_current = airways_mask.copy()
    thresholds = [-890, -400, 0]
    for thr in thresholds:
        next_frontier = np.zeros_like(extended)
        dilated = binary_dilation(frontier_current, iterations=1)
        frontier = dilated & (~extended)
        for z, y, x in np.argwhere(frontier):
            if final_roi[z, y, x] > 0 and vol_clip[z, y, x] <= thr:
                extended[z, y, x] = 1
                next_frontier[z, y, x] = 1
        frontier_current = next_frontier
        if not np.any(frontier_current):
            break

    return extended.astype(np.uint8), len(seed_points)

# --- Función Dice optimizada ---
def dice_coefficient(y_true, y_pred):
    y_true_sum = int(np.sum(y_true))
    y_pred_sum = int(np.sum(y_pred))
    if y_true_sum + y_pred_sum == 0:
        return 1.0
    intersection = int(np.sum(y_true[y_pred == 1]))
    return (2. * intersection) / (y_true_sum + y_pred_sum)

# --- Procesamiento con gestión de memoria agresiva ---
existing = {f for f in os.listdir(AIRWAYS_SUBDIR) if f.endswith(".nii.gz")}
to_process = [vid for vid in loaded_data.keys() if f"{vid}.nii.gz" not in existing]
print(f"🔄 Procesando {len(to_process)} volúmenes con Cell 4 TUBULAR...")

# --- CSV setup ---
header = ["volume_id", "dice", "time_seconds", "voxels", "components"]
write_header = not os.path.exists(METRICS_FILE)
if write_header:
    with open(METRICS_FILE, "w", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(header)

# --- Bucle principal con gestión de memoria ---
for vol_id in tqdm(to_process, desc="Cell 4 TUBULAR"):
    try:
        start_time = time.time()
        
        # Cargar datos de forma eficiente
        data = loaded_data[vol_id]
        vol_orig, affine, header_nib = load_volume_efficient(data["img_path"])
        label_manual = load_label_efficient(data["lbl_path"])
        vol_clip = np.clip(vol_orig, HU_MIN, HU_MAX)

        # Cargar ROI
        roi_path = os.path.join(ROIS_DIR, f"{vol_id}.nii.gz")
        if not os.path.exists(roi_path):
            print(f"\n⚠️  ROI no encontrada para {vol_id}. Saltando...")
            continue
        final_roi = load_label_efficient(roi_path)

        # Segmentar
        airways_mask, n_seeds = segment_airways_tubular(vol_clip, final_roi)

        # Métricas
        dice = dice_coefficient(label_manual, airways_mask)
        voxels = int(airways_mask.sum())
        components = cc_label(airways_mask)[1]
        processing_time = time.time() - start_time

        # Guardar resultado
        output_path = os.path.join(AIRWAYS_SUBDIR, f"{vol_id}.nii.gz")
        nib.save(nib.Nifti1Image(airways_mask, affine, header_nib), output_path)

        # Registrar métricas
        with open(METRICS_FILE, "a", newline="") as f:
            writer = csv.writer(f)
            writer.writerow([vol_id, f"{dice:.4f}", f"{processing_time:.2f}", voxels, components])

        # LIMPIEZA AGRESIVA DE MEMORIA
        del vol_orig, vol_clip, label_manual, final_roi, airways_mask, data
        gc.collect()
        
        # Forzar liberación de memoria (opcional, para sistemas con poca RAM)
        if hasattr(gc, 'collect_generations'):
            gc.collect_generations()
        else:
            gc.collect()

    except MemoryError:
        print(f"\nMemoryWarning Memoria insuficiente para {vol_id}. Saltando...")
        gc.collect()
        continue
    except Exception as e:
        print(f"\n❌ Error en {vol_id}: {e}")
        gc.collect()
        continue

print("✅ Cell 4 TUBULAR aplicada a todos los volúmenes. Métricas en 'airways_metrics.csv'.")

🔄 Procesando 0 volúmenes con Cell 4 TUBULAR...


Cell 4 TUBULAR: 0it [00:00, ?it/s]

✅ Cell 4 TUBULAR aplicada a todos los volúmenes. Métricas en 'airways_metrics.csv'.


In [9]:
# ---------------------------------------------------------------
# Celda de Evaluación: Validación de Labels Automáticos (Full ROI)
# Verifica la calidad de los labels generados por el pipeline tubular
# ---------------------------------------------------------------

import os
import pandas as pd
import numpy as np
import nibabel as nib
from tqdm import tqdm

# Rutas
METRICS_FILE = "./airways_metrics.csv"
AIRWAYS_AUTO_DIR = "./airways_auto"
MANUAL_LABELS_DIR = "./TrainBatch1/labelsTr"

# Validación de archivo de métricas
if not os.path.exists(METRICS_FILE):
    raise FileNotFoundError(f"Archivo de métricas no encontrado: {METRICS_FILE}")

# Carga de métricas
metrics_df = pd.read_csv(METRICS_FILE)
print(f"Métricas cargadas: {len(metrics_df)} casos")

# Estadísticas descriptivas
dice_mean = metrics_df['dice'].mean()
dice_std = metrics_df['dice'].std()
voxels_mean = metrics_df['voxels'].mean()
voxels_std = metrics_df['voxels'].std()

print(f"\nEstadísticas generales:")
print(f"- Dice promedio: {dice_mean:.4f} ± {dice_std:.4f}")
print(f"- Vóxeles promedio: {voxels_mean:,.0f} ± {voxels_std:,.0f}")
print(f"- Rango Dice: [{metrics_df['dice'].min():.4f}, {metrics_df['dice'].max():.4f}]")

# Filtrado por calidad
HIGH_QUALITY_THRESHOLD = 0.75
high_quality_df = metrics_df[metrics_df["dice"] > HIGH_QUALITY_THRESHOLD]
print(f"\nCasos de alta calidad (Dice > {HIGH_QUALITY_THRESHOLD}): {len(high_quality_df)}")
print(f"- Dice promedio (alta calidad): {high_quality_df['dice'].mean():.4f}")
print(f"- Vóxeles promedio (alta calidad): {high_quality_df['voxels'].mean():,.0f}")

# Verificación de emparejamiento
def extract_case_id(filename):
    """Extrae case_id de diferentes formatos."""
    name = filename.replace(".nii.gz", "")
    if "_0000" in name:
        return name.replace("_0000", "")
    return name

auto_ids = set(extract_case_id(f) for f in os.listdir(AIRWAYS_AUTO_DIR) if f.endswith(".nii.gz"))
manual_ids = set(extract_case_id(f) for f in os.listdir(MANUAL_LABELS_DIR) if f.endswith(".nii.gz"))
common_ids = auto_ids & manual_ids

print(f"\nVerificación de emparejamiento:")
print(f"- Casos automáticos: {len(auto_ids)}")
print(f"- Casos manuales: {len(manual_ids)}")
print(f"- Casos emparejados: {len(common_ids)}")

# Validación final
valid_high_quality = high_quality_df[high_quality_df["volume_id"].isin(common_ids)]
print(f"\nCasos válidos de alta calidad para entrenamiento: {len(valid_high_quality)}")

# Guardado de lista
HIGH_QUALITY_CASES_FILE = "./high_quality_cases.txt"
with open(HIGH_QUALITY_CASES_FILE, 'w') as f:
    for case_id in sorted(valid_high_quality["volume_id"]):
        f.write(f"{case_id}\n")

# Variables globales para Cell 1
high_quality_case_set = set(valid_high_quality["volume_id"])
print(f"\nPreparación completada. Lista guardada en: {HIGH_QUALITY_CASES_FILE}")

Métricas cargadas: 350 casos

Estadísticas generales:
- Dice promedio: 0.5921 ± 0.2841
- Vóxeles promedio: 124,313 ± 69,022
- Rango Dice: [0.0000, 0.8931]

Casos de alta calidad (Dice > 0.75): 115
- Dice promedio (alta calidad): 0.7928
- Vóxeles promedio (alta calidad): 165,846

Verificación de emparejamiento:
- Casos automáticos: 150
- Casos manuales: 150
- Casos emparejados: 150

Casos válidos de alta calidad para entrenamiento: 115

Preparación completada. Lista guardada en: ./high_quality_cases.txt
